In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/caill/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
help(yelp_api.search_query)

Help on method search_query in module yelpapi.yelpapi:

search_query(**kwargs) method of yelpapi.yelpapi.YelpAPI instance
    Query the Yelp Search API.
    
    documentation: https://www.yelp.com/developers/documentation/v3/business_search
    
    required parameters:
        * one of either:
            * location - text specifying a location to search for
            * latitude and longitude



In [4]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='GA, GA',
                                       term='Pasta')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
LOCATION = 'GA, GA'
TERM = 'Pasta'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_GA_pasta.json"
JSON_FILE

'Data/results_in_progress_GA_pasta.json'

In [7]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)           
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [8]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_GA_pasta.json already exists. Deleting previous file...
[i] Data/results_in_progress_GA_pasta.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [9]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [10]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,eMJGDVzTH7mbGNWIfz6jcQ,la-tavola-trattoria-atlanta,La Tavola Trattoria,https://s3-media4.fl.yelpcdn.com/bphoto/EeyMhJ...,False,https://www.yelp.com/biz/la-tavola-trattoria-a...,599,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 33.7824462, 'longitude': -84.3549...",[delivery],$$,"{'address1': '992 Virginia Ave NE', 'address2'...",+14048735430,(404) 873-5430,6270.495216
1,ERVKNes7rXoQg4J0gWhMdg,forza-storico-atlanta,Forza Storico,https://s3-media3.fl.yelpcdn.com/bphoto/MSNL6R...,False,https://www.yelp.com/biz/forza-storico-atlanta...,404,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.78662558409769, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1198 Howell Mill Rd', 'address2'...",+14044648096,(404) 464-8096,2251.206980
2,uO4A77jXMPfZmlbf_52zRw,boccalupo-atlanta,BoccaLupo,https://s3-media2.fl.yelpcdn.com/bphoto/txg_k6...,False,https://www.yelp.com/biz/boccalupo-atlanta?adj...,652,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 33.7547495778871, 'longitude': -8...",[delivery],$$,"{'address1': '753 Edgewood Ave NE', 'address2'...",+14045772332,(404) 577-2332,5504.599927
3,kXsvTG6zzQt0xKP8jFUpMQ,mirko-pasta-atlanta,Mirko Pasta,https://s3-media3.fl.yelpcdn.com/bphoto/X1H17e...,False,https://www.yelp.com/biz/mirko-pasta-atlanta?a...,294,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.87007, 'longitude': -84.38057}","[delivery, pickup]",$$,"{'address1': '4233 Roswell Rd NE', 'address2':...",+14043032110,(404) 303-2110,11973.167598
4,snPiykJNsT51YSf3Rpm2CA,mezza-luna-smyrna,Mezza Luna,https://s3-media3.fl.yelpcdn.com/bphoto/jaoVKp...,False,https://www.yelp.com/biz/mezza-luna-smyrna?adj...,649,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 33.882797, 'longitude': -84.504883}","[delivery, pickup]",$$,"{'address1': '1669 Spring Rd Se', 'address2': ...",+17703190333,(770) 319-0333,15007.002640


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,H41Pqjfv21BJwPvWmW7ohA,sprouts-farmers-market-marietta,Sprouts Farmers Market,https://s3-media3.fl.yelpcdn.com/bphoto/tq8NXa...,False,https://www.yelp.com/biz/sprouts-farmers-marke...,40,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",2.5,"{'latitude': 33.98185903, 'longitude': -84.431...",[],$$,"{'address1': '4101 Roswell Rd', 'address2': 'S...",+17702009172,(770) 200-9172,23852.677457
996,is5vDRt1DVD8v_ETWz2yCg,jasons-deli-lawrenceville,Jason's Deli,https://s3-media4.fl.yelpcdn.com/bphoto/BEVfZf...,False,https://www.yelp.com/biz/jasons-deli-lawrencev...,123,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.5,"{'latitude': 33.9678709921065, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '918 Duluth Hwy', 'address2': '',...",+17709629090,(770) 962-9090,42788.774816
997,QN39JJl_sCq0P55HHEEohQ,chicken-salad-chick-of-alpharetta-alpharetta-2,Chicken Salad Chick of Alpharetta,https://s3-media2.fl.yelpcdn.com/bphoto/b7NpG7...,False,https://www.yelp.com/biz/chicken-salad-chick-o...,132,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 34.04966076708913, 'longitude': -...","[delivery, pickup]",$,"{'address1': '970 N Point Dr', 'address2': 'St...",+17703366654,(770) 336-6654,33779.164305
998,yMKQoXFFsDk1IfgXxYfafQ,jasons-deli-tucker,Jason's Deli,https://s3-media1.fl.yelpcdn.com/bphoto/WSxoIU...,False,https://www.yelp.com/biz/jasons-deli-tucker?ad...,149,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.0,"{'latitude': 33.8452486525497, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '4073 Lavista Rd', 'address2': ''...",+17704934020,(770) 493-4020,17777.727252
999,QTanooGEknqEWNRMsgIHaA,zaxbys-chicken-fingers-and-buffalo-wings-atlan...,Zaxby's Chicken Fingers & Buffalo Wings,https://s3-media1.fl.yelpcdn.com/bphoto/4c_4xv...,False,https://www.yelp.com/biz/zaxbys-chicken-finger...,54,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"{'latitude': 33.72858715, 'longitude': -84.573...","[delivery, pickup]",$$,"{'address1': '925 Camp Fulton Pkwy SW', 'addre...",+14046296770,(404) 629-6770,14819.878160


In [11]:
# check for duplicate ID's 
#final_df.duplicated(subset='id').sum()

## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [13]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_GA_pasta.csv.gz', compression='gzip',index=False)